In [1]:
import os
import re
import pandas as pd
import numpy as np
from natsort import natsorted, index_natsorted, order_by_index

part_pattern = "(^|ст(\.|))\s*\d{3}(\.\d{1}|)(\s*ч*\.*\s*\d{1}|)(\s*\.\d{1}|)"

def dropFirstRows(df, value2search, rowsNum = None):
    """
    Функция находит строку первого столбца в таблице, значение которой соответствует параметру value2search,
    и удаляет количество строк, указанное в параметре rowsNum.
    Если rowsNum не задан - удаляются все строки начиная с самой верхней.
    """
    for index, val in df.iloc[:,0].str.contains(value2search).iteritems():
        if val == True and val != np.nan and index !=0:
            if rowsNum is None:
                df.drop(df.index[0:index], inplace=True)
            else:
                df.drop(df.index[index-rowsNum:index], inplace=True)
                break
            df.reset_index(drop=True, inplace=True)
    return df

def cutNumbers(string):
    '''Служебная функция нужная, чтобы при склеивании строк в tackleMergedCells вырезать строки, содержащие только числа'''
    string = str(string)
    string = string.replace("nan", "")
    match = re.search(r'(^\d+$)', string)
    if match:
        string = string.replace(match.group(), "")
    return string



def deleteUnusedCols(df, year):
    """
    Удаляет пустые колонки, а также неиспользуемые нами колонки
    """
    df.dropna(axis='columns', how='all', inplace=True) # удаляем пустые колонки
    df.columns = pd.Series(df.iloc[0,]).astype('str') # превращаем первую строку в заголовок таблицы
    df = cleanСolsNames(df)

    indeces2remove = []

    # находим номер колонки, в которой содержится ненужный нам номер строки или пункта, если они попали в заголовок таблицы
    i = -1
    for col in df.columns:
        i += 1
        if col.find('№') > -1:
            indeces2remove.append(i)
            break
    # или находим номер колонки, в которой содержится ненужный нам номер строки или пункта, если они не попали в заголовок и отличаются от других колонок двумя строками с NA
    header_rows = 3
    j = -1
    for index, col in df.iloc[0:header_rows].iteritems():
        j += 1
        if col.isnull().values.all():
            indeces2remove.append(j)

    # удаляем колонку с найденным индексом
    column_numbers = [x for x in range(df.shape[1])]  # list of columns' integer indices
    for index in indeces2remove:
        column_numbers.remove(index) #removing column integer index i
    df = df.iloc[:, column_numbers]
    df.columns = range(df.shape[1])
    return df

def cleanСolsNames(df):
    '''Очищает заголовок таблицы символов перевода строки, двоеточий, переводит заглавные буквы в строчные'''
    df.columns = df.columns.str.strip()
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace("\n", "")
    df.columns = df.columns.str.replace(":", "")
    return df

def renameColumns(df, column_mapping):
    rename_dict = dict(zip(column_mapping['original'], column_mapping['translation']))
    df.rename(columns=rename_dict, inplace=True)
    # проверяем, что все колонки переименовались
    print('Проверяем, что все переименовано.\nЕсли ничего не печатается, то все переименовано.\nЕсли остались не переименованные колонки, это иногда ок: мы не используем колонки, значения в которых легко вычисляются сложнением других колонок или повторяются в других файлах')
    for column in df.columns:
        if re.search(r'[а-яА-Я]', str(column)):
            print(column)
    return df

def dropNARows(df):
    # Preview the first row that would be dropped
    rows_to_drop_first = df.iloc[0:1]
    #if not rows_to_drop_first.empty:
      #print("Rows that will be dropped (first row):")
      #print(rows_to_drop_first)
      #print(rows_to_drop_first.clause)

    # Preview rows with NaN values that would be dropped
    rows_to_drop_na = df[df.isna().any(axis=1)]
    #if not rows_to_drop_na.empty:
      #print(f"\nRows with NaN values that will be dropped ({len(rows_to_drop_na)} rows):")
      #print(rows_to_drop_na)
      #print(rows_to_drop_na.clause)

    df.drop(df.index[0:1], inplace=True)
    df.dropna(axis='rows', how='any', inplace=True)
    return df

def deleteUselessRows(df, year):
    '''Удаляем строки, в которых нет информации о конкретных статьях'''
    df['clause'] = df['clause'].astype(str)
    if 'name' in df.columns:
        df = df[~df['name'].str.contains('ВСЕГО|ИТОГО')]
    df = df.reset_index(drop=True)
    return df

def insertEmptyRows(df, year, clauses2Insert):
    '''Поскольку в 2012 и 2015 годах встречаются строки, в которых собраны несколько статей, все значения по которым равны
    нулю, т.е. не было ни осужденных, ни прекращенных дел, ни оправданных и т.д., то такие строки можно разбить по статьям
    '''
    year = str(year)
    dict2Insert = {}
    if year in clauses2Insert.keys():
        for clause in clauses2Insert[year]:
            for col in df.columns:
                dict2Insert[col] = [0]
            dict2Insert['clause'] = clause
            df2Insert = pd.DataFrame.from_dict(dict2Insert)
            df_concat = pd.concat([df, df2Insert], sort=True)
            df = df_concat.reset_index(drop=True)
    return df

def cleanClauseCol(df):
    df['clause'] = df['clause'].str.replace("Составы преступлений, введенные в УК РФ после утверждения форм отчетности:\nКлевета \(введ. ФЗ от 28.07.2012 N 141-ФЗ\) ст. ", "")
    df['clause'] = df['clause'].str.replace("Составы преступлений, введенные в раздел IX \"Преступления против общественной безопасности и общественного порядка\" УК РФ после утверждения форм отчетности приказом № 127\s+ст. ", "")
    df['clause'] = df['clause'].str.replace("Клевета в отношении судьи, присяжного заседателя, прокурора, следователя, лица, производящего дознание, судебного пристава \(введ. ФЗ от 28.07.2012 N 141-ФЗ\)\s+ст. ", "")

    df['clause'] = df['clause'].str.replace('Мошенничество, совершенное в сфере кредитования; при получении выплат; с использованием платежных карт; в сфере предпринимательской деят-ти; в сфере страхования; в сфере компьютерной информации \(введ. ФЗ от 29.11.2012 N 207-ФЗ\) ст\. ', '')

    df['clause'] = df['clause'].astype("str")
    df['clause'] = df['clause'].str.replace(" ", "")
    df['clause'] = df['clause'].str.replace("-", ".")
    df['clause'] = df['clause'].str.replace("ст(\.|)", "")
    df['clause'] = df['clause'].str.replace("\.ч", "ч")

    df['clause'] = df['clause'].str.replace("207\(.+\),", "")
    return df

def clauses2column(df):
    # создаем колонку part с номером статьи и части без примечаний и удаляем из нее лишние пробелы
    # df['clause'] = df['clause'].str.replace("\.\s*ч", "ч")
    # df['part'] = df.apply(lambda row: re.search(part_pattern, row['clause']).group(), axis=1)
    # на случай, если все-таки захочется вернуть воинские преступления
    df['part'] = df.apply(lambda row: re.search(part_pattern, row['clause']).group() if re.search(r'[Вв]оинск', row['clause']) is None else "Воинские преступления", axis=1)
    df['part'] = df['part'].str.replace("^ст\.\s", "")
    df['part'] = df['part'].str.replace(" ", "")
    # тут мы отрезаем номер статьи от примечаний типа "в старой редакции" - может пригодиться для целей сверки таблиц, но в целом нет
    '''
    df['clause'] = df.apply(lambda row: row['clause'].replace(re.search(part_pattern, row['clause']).group(), ""), axis=1)
    df['clause'] = df['clause'].str.replace("\sст. $", "")
    df['clause'] = df['clause'].str.replace("^\s", "")
    # колонку clause переименовываем в comments
    df.rename(index=str, columns={"clause": "comments"}, inplace=True)
    '''
    # в колонку clause складываем статьи без частей, используя ранее созданную колонку part
    df['clause'] = df.apply(lambda row: re.search(r'(\d{3}(\.\d{1}|))', row['part']).group() if re.search(r'[Вв]оинск', row['clause']) is None else "Воинские преступления", axis=1)
    #df['clause'] = df.apply(lambda row: re.search(r'(\d{3}(\.\d{1}|))', row['part']).group(), axis=1)
    df.replace("Воинскиепреступления", "Воинские преступления", inplace=True)
    return df

def addMilitaryOfences(df):
    df = pd.concat([df, pd.DataFrame([[np.nan] * df.shape[1]], columns=df.columns)], ignore_index=True)
    ind = np.where(df['name'].isna())[0][0]
    df.at[ind, 'name'] = "Воинские преступления"
    df.at[ind, 'clause'] = "Воинские преступления"
    df.at[ind, 'part'] = "Воинские преступления"
    return df

def rearrangeCols(df, firstCols):
    cols = firstCols  + [col for col in df.columns.tolist() if col not in firstCols]
    df = df[cols]
    return df

def sortTable(df):
    '''Сортирует таблицу по колонке part - со статьей и часть методом натуральной сортировки'''
    df = df.reindex(index=order_by_index(df.index, index_natsorted(df['part'])))
    df = df.reset_index(drop=True)
    return df

def tackleMergedCells(df):
    df.iloc[0:1].fillna(method='ffill', axis='columns', inplace=True)
    df.iloc[0:3].replace(to_replace=['\n', '-', '\*', '\s{3}', '\s{2}'],value=[' ', '', '', ' ', ' '], regex=True, inplace=True)
    for i in range(2, len(df.iloc[0])): # 2 - это номер колонки, с которой начинается смерживание объединенных колонок
        df.at[0, i] = str(df.iloc[0][i]).replace("nan", "") + " " + cutNumbers(df.iloc[1][i]) + " " + cutNumbers(df.iloc[2][i])

    df.columns = pd.Series(df.iloc[0,]).str.strip()
    df.reset_index(drop=True, inplace=True)
    return df

def nameSeparatedRows(df, names):
    for key in names.keys():
        ind = df.index[df['part'] == key].tolist()
        if len(ind) == 1:
            df.at[ind[0], 'name'] = names[key]
        else:
          print(f'Строка с частью {key} повторяется {len(ind)} раз, индексы {ind}')
    return df

def meltTable(df, year):
    '''Приводит таблицу в длинную форму'''
    df.insert(0, 'year', [year]*len(df))
    df = pd.melt(df, id_vars=['year', 'clause', 'part', 'name'], value_vars = list(df.columns)[4:], var_name='parameter', value_name='value')
    return df

# Проверочные функции

def checkTablesLen(main, add, parameters = None):
    if parameters is not None:
        if len(main) != len(parameters):
            print("Не ОК: длины таблиц № 10.3 и № 10.3.1 не совпадают")
        else:
            print("ОК: длины таблиц № 10.3 и № 10.3.1 совпадают")
    if len(main) != len(add):
        print("Не ОК: длины таблиц № 10.3 и № 10-а не совпадают")
    else:
        print("ОК: длины таблиц № 10.3 и № 10-а совпадают")

def checkNumbersBetweenForms(year, mainDF, addDF, parametersDF = None):
    if year > 2010:
        for i in range(len(mainDF)):
            if mainDF.iloc[i]['part'] != parametersDF.iloc[i]['part']:
                print(i, "main", mainDF.iloc[i]['part'], "parameters", parametersDF.iloc[i]['part'])

            imprisonment_sum = parametersDF.iloc[i]['primaryImprisonment1'] + \
                                                         parametersDF.iloc[i]['primaryImprisonment1_2'] + \
                                                         parametersDF.iloc[i]['primaryImprisonment2_3'] + \
                                                         parametersDF.iloc[i]['primaryImprisonment3_5'] +\
                                                         parametersDF.iloc[i]['primaryImprisonment5_8'] +\
                                                         parametersDF.iloc[i]['primaryImprisonment8_10'] +\
                                                         parametersDF.iloc[i]['primaryImprisonment10_15'] +\
                                                         parametersDF.iloc[i]['primaryImprisonment15_20']
            if mainDF.iloc[i]['primaryImprisonment'] != imprisonment_sum:
                print("Не совпадает число осужденных к лишению свободы.", "\nСтатья: ", mainDF.iloc[i]['part'], "\n10.3", "Лишение свободы всего: ", mainDF.iloc[i]['primaryImprisonment'],
                      "\n10.3.1", "Лишение свободы сумма: ", imprisonment_sum)

            if mainDF.iloc[i]['part'] != addDF.iloc[i]['part']:
                print("Не совпадают номера статей. 10.3:", mainDF.iloc[i]['part'], "10-a:", addDF.iloc[i]['part'], "Год: ", year)
            if mainDF.iloc[i]['totalConvicted'] != addDF.iloc[i]['totalConvictedMain']:
                print("Не совпадает число осужденных по основной статье.", "\nСтатья: ", mainDF.iloc[i]['part'], "\n10.3:", mainDF.iloc[i]['totalConvicted'], "\n10-a:", addDF.iloc[i]['totalConvictedMain'], "\nГод:", year)
    else:
        for i in range(len(mainDF)):
            if mainDF.iloc[i]['part'] != addDF.iloc[i]['part']:
                print("Не совпадают номера статей. 10.3:", mainDF.iloc[i]['part'], "10-a:", addDF.iloc[i]['part'], "Год: ", year)
            if mainDF.iloc[i]['totalConvicted'] != addDF.iloc[i]['totalConvictedMain']:
                print("Не совпадает число осужденных по основной статье.", "\nСтатья: ", mainDF.iloc[i]['part'], "\n10.3:", mainDF.iloc[i]['totalConvicted'], "\n10-a:", addDF.iloc[i]['totalConvictedMain'], "\nГод:", year)

def compareSums(df, columns, total_values, start):
    columns = list(columns)[start:]
    cols = [col for col in columns if not re.search(r'[а-яА-Я]', col)]
    values2remove = []
    for col in columns:
        match = re.search(r'[а-яА-Я]{2,}', col)
        if match:
            ind = columns.index(col)
            values2remove.append(total_values[ind])
    values = [val for val in total_values if val not in values2remove]
    total_values = values
    DFreordered = df[cols]
    sums = DFreordered.sum(axis='index')
#     print(sums)
    if len(total_values) != len(sums):
        print("Количество колонок не совпадает")
    for i in range(0, len(total_values)):
        if total_values[i] != sums[i]:
            print('\nСумма значений в колонке', DFreordered.columns[i], sums[i], '\n', 'Значение в строке "Всего по составам УК РФ" ', total_values[i])

In [2]:
import os
import re
import pandas as pd
import numpy as np
from natsort import natsorted, index_natsorted, order_by_index

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 1000)

import warnings
warnings.filterwarnings('ignore')

In [3]:
year = 2022

Обработка файлов формы № 10.3

In [4]:
file_path = "ExcelFiles/Main/K7-svod_vse_sudy-2023.xls"

mainDF = pd.read_excel(file_path, sheet_name=1)

# Убираем строки и колонки, не содержащие полезной информации
mainDF = dropFirstRows(mainDF, 'Статьи УК РФ')
mainDF = deleteUnusedCols(mainDF, year)

# эта строка будет нужна позже для проверки
total_values = list(mainDF[mainDF.iloc[:,0].astype('str').str.contains("Всего по составам УК РФ|Всего лиц по составам УК РФ")].values[0][1:])

mainDF = dropFirstRows(mainDF, '105 ч. 1|105 ч.1|105ч. 1', 2)

# Переводим названия колонок
mainDF.iloc[1][16] = mainDF.iloc[1][15]
mainDF.iloc[1][18] = mainDF.iloc[1][17]
mainDF.iloc[1][20] = mainDF.iloc[1][19]
mainDF = tackleMergedCells(mainDF)
mainDF = cleanСolsNames(mainDF)

columns = mainDF.columns # эта строка тоже будет нужна позже для проверки

columns2eng = pd.read_csv('translations/2022/colNames2engNames_2022.csv')
columns2eng.replace('\(\d{4}\)', '', inplace=True, regex=True)
mainDF = renameColumns(mainDF, columns2eng)

# Убираем строки, не содержащие полезной информации
mainDF = dropNARows(mainDF)
mainDF = deleteUselessRows(mainDF, year)

mainDF = cleanClauseCol(mainDF) # чистим колонку с номерами статей от лишнего текста
mainDF = clauses2column(mainDF) # используя регулярное выражение, создаем колонки со номерами статей и частей

firstCols = ['clause', 'part', 'totalConvicted']
mainDF = rearrangeCols(mainDF, firstCols) # меняем порядок столбцов для удобства
mainDF = sortTable(mainDF) # упорядочиваем строки

Проверяем, что все переименовано.
Если ничего не печатается, то все переименовано.
Если остались не переименованные колонки, это иногда ок: мы не используем колонки, значения в которых легко вычисляются сложнением других колонок или повторяются в других файлах


In [5]:
compareSums(mainDF, columns, total_values, 2)

In [6]:
# Для проверки, что никакие цифры не потерялись, можно посчитать суммы по столбцам и выборочно сверить с исходной таблицей (строка ВСЕГО ПО СОСТАВАМ).

mainDF.sum(axis=0)

0
clause                                   105105106107107108108109109109110.1110.1110.11...
part                                     105ч.1105ч.2106107ч.1107ч.2108ч.1108ч.2109ч.11...
totalConvicted                                                                      555743
primaryLifeSentence                                                                     39
primaryImprisonment                                                                 164702
primarySuspended                                                                    131029
primaryMilitaryDisciplinaryUnit                                                         46
primaryArrest                                                                            9
primaryRestrain                                                                      23789
primaryRestrictionsInMilitaryService                                                    96
primaryCorrectionalLabour                                                            475

In [7]:
mainDF.head()

,clause,part,totalConvicted,primaryLifeSentence,primaryImprisonment,primarySuspended,primaryMilitaryDisciplinaryUnit,primaryArrest,primaryRestrain,primaryRestrictionsInMilitaryService,primaryCorrectionalLabour,primaryCommunityService,primaryForcedLabour,primaryDisqualification,primaryFine,primaryOther,exemptionTimeServedFromImprisonment,exemptionTimeServedOther,exemptionAmnestyFromImprisonment,exemptionAmnestyOther,exemptionOtherGroundsFromImprisonment,exemptionOtherGroundsOther,acquittal,dismissalAbsenceOfEvent,dismissalAmnesty,dismissalReconciliation,dismissalRepentance,dismissalCourtFine,dismissalOther,dismissalRepentance2,coerciveMeasures,addDisqualification,addFine,addTitlesWithdraw,addRestrain,unfinishedOffence,noCrimeSelf-defence,noCrimeNecessity,noCrimeOther
0,105,105ч.1,4461,0,4424,20,0,0,1,0,0,0,0,0,0,0,0,0,0,0,14,2,32,2,1,0,0,0,350,0,400,3,0,17,455,801,1,0,0
1,105,105ч.2,1225,33,1187,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,3,0,23,1,0,0,0,0,62,0,110,4,0,2,806,295,0,0,0
2,106,106,22,0,14,2,0,0,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3,0,0,0,0,6,0,0,0
3,107,107ч.1,27,0,1,0,0,0,15,0,3,0,0,0,0,3,1,3,0,0,0,1,0,0,0,3,0,0,2,0,1,0,0,0,0,0,0,0,0
4,107,107ч.2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Обработка файлов формы № 10.3.1

In [8]:
parametersFile = "ExcelFiles/MainParameters/K7_1-svod_vse_sudy-2023.xls"
parametersDF = pd.read_excel(parametersFile, 1, header=None)

total_values = list(parametersDF[parametersDF.iloc[:,0].astype('str').str.contains("Всего по составам УК РФ|Всего лиц по составам УК РФ")].values[0][3:])

parametersDF = dropFirstRows(parametersDF, 'Статьи УК РФ')
parametersDF = deleteUnusedCols(parametersDF, year)

parametersDF = dropFirstRows(parametersDF, '105 ч. 1|105 ч.1|105ч. 1', 2)

parametersDF = tackleMergedCells(parametersDF)

parametersDF = cleanСolsNames(parametersDF)
columns2eng = pd.read_csv('translations/2023/colNames2engNamesParameters_2023.csv')
columns2eng.replace('\(\d{4}\)', '', inplace=True, regex=True)
parametersDF = renameColumns(parametersDF, columns2eng)

columns = parametersDF.columns

parametersDF = parametersDF[parametersDF.columns[~parametersDF.columns.str.contains('[а-яА-Я]{2,}', regex=True)]]
parametersDF = dropNARows(parametersDF)
parametersDF = deleteUselessRows(parametersDF, year)

parametersDF = cleanClauseCol(parametersDF)
parametersDF = clauses2column(parametersDF)

firstCols = ['clause', 'part']
parametersDF = rearrangeCols(parametersDF, firstCols)
parametersDF = sortTable(parametersDF)

Проверяем, что все переименовано.
Если ничего не печатается, то все переименовано.
Если остались не переименованные колонки, это иногда ок: мы не используем колонки, значения в которых легко вычисляются сложнением других колонок или повторяются в других файлах
лишение свободы всего (число лиц)
общая сумма штрафов (из гр. 13 "основное наказание" ф. 10.3) всего лиц
общая сумма штрафов (из гр. 28 "дополнительное наказание" ф. 10.3) всего лиц
общая сумма судебных штрафов, назначенных в соответствии со статьей 104.4 ук рф всего лиц


In [9]:
# Незначительные расхождения это ок, т.к. скорее всего в исходных формах они тоже есть (мир суддепа несовершенен!)

compareSums(parametersDF, columns, total_values, 2)


Сумма значений в колонке primaryImprisonment3_5 18784 
 Значение в строке "Всего по составам УК РФ"  18785

Сумма значений в колонке primaryImprisonment5_8 18319 
 Значение в строке "Всего по составам УК РФ"  18318

Сумма значений в колонке primaryFineSum 4547015023 
 Значение в строке "Всего по составам УК РФ"  4546902023

Сумма значений в колонке addFineSum 3704568636 
 Значение в строке "Всего по составам УК РФ"  3886326468

Сумма значений в колонке dismissalCourtFine5 2938 
 Значение в строке "Всего по составам УК РФ"  2930

Сумма значений в колонке dismissalCourtFine5_25 9255 
 Значение в строке "Всего по составам УК РФ"  9249

Сумма значений в колонке dismissalCourtFine25_100 2522 
 Значение в строке "Всего по составам УК РФ"  2523

Сумма значений в колонке dismissalCourtFine100 252 
 Значение в строке "Всего по составам УК РФ"  254

Сумма значений в колонке dismissalCourtFineSum 302273461 
 Значение в строке "Всего по составам УК РФ"  301862461


In [10]:
parametersDF.sum(axis=0)

0
clause                                105105106107107108108109109109110.1110.1110.11...
part                                  105ч.1105ч.2106107ч.1107ч.2108ч.1108ч.2109ч.11...
primaryImprisonment1                                                              56150
primaryImprisonment1_2                                                            38943
primaryImprisonment2_3                                                            19536
primaryImprisonment3_5                                                            18784
primaryImprisonment5_8                                                            18319
primaryImprisonment8_10                                                            7853
primaryImprisonment10_15                                                           4460
primaryImprisonment15_20                                                            657
primaryImprisonmentUnderLowerLimit                                                 7867
primaryFine5                  

In [11]:
parametersDF.tail()

,clause,part,primaryImprisonment1,primaryImprisonment1_2,primaryImprisonment2_3,primaryImprisonment3_5,primaryImprisonment5_8,primaryImprisonment8_10,primaryImprisonment10_15,primaryImprisonment15_20,primaryImprisonmentUnderLowerLimit,primaryFine5,primaryFine5_25,primaryFine25_100,primaryFine100_300,primaryFine300_500,primaryFine500_1M,primaryFine1M,addFine5,addFine5_25,addFine25_100,addFine100_300,addFine300_500,addFine500_1M,addFine1M,primaryFineSum,addFineSum,dismissalCourtFine5,dismissalCourtFine5_25,dismissalCourtFine25_100,dismissalCourtFine100,dismissalCourtFineSum
1047,360,360ч.2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1048,361,361ч.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1049,361,361ч.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1050,361,361ч.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1051,Воинские преступления,Воинские преступления,106,405,420,750,552,4,1,0,257,0,47,84,1,1,0,0,0,0,0,0,0,0,0,5213000,0,2,21,10,0,770000


Обработка файлов формы № 10-а

In [12]:
addFile = 'ExcelFiles/Add/k3-svod_vse_sudy-2023.xls'

df_list = pd.read_excel(addFile, sheet_name=None, header = None)
dfs = []
for key in list(df_list.keys())[2:-3]: # -3 мы исключаем листы, где приводятся данные по пунктам статей, разберемся с ними отдельной задачей
    df = df_list[key]
    df = dropFirstRows(df, 'Виды преступлений')
    df = dropFirstRows(df, '[а-яА-Я]{3,}', 1)
    df = deleteUnusedCols(df, year)
    df = tackleMergedCells(df)
    columns2eng = pd.read_csv('translations/2023/colNames2engNamesAdd_2023.csv')
    columns2eng.replace('\(\d{4}\)', '', inplace=True, regex=True)
    df = renameColumns(df, columns2eng) # тут заголовок таблицы предварительно чистить не надо

    df = df[df.columns[~df.columns.str.contains('[а-яА-Я]{2,}', regex=True)]]
    df = dropNARows(df)
    dfs.append(df)
    addDF = pd.concat(dfs, ignore_index=True)

addDF.clause = addDF.clause.astype(str)

addDF = cleanClauseCol(addDF)
addDF = clauses2column(addDF)
addDF = addMilitaryOfences(addDF)

addDF.clause = addDF.clause.astype(str)
addDF.name = addDF.name.astype(str)

names = {
'172.4': 'Незаконное осуществление деятельности по возврату просроченной задолженности физических лиц',
'260.1': 'Умышленные уничтожение или повреждение, а равно незаконные добыча, сбор и оборот особо ценных растений и грибов, принадлежащих к видам, занесенным в Красную книгу Российской Федерации и (или) охраняемым международными договорами Российской Федерации',
'330.3': 'Осуществление деятельности на территории Российской Федерации иностранной или международной некоммерческой неправительственной организации, сведения о структурных подразделениях которой отсутствуют в реестре филиалов и представительств международных организаций и иностранных некоммерческих неправительственных организаций или которая не имеет зарегистрированного в порядке, предусмотренном законодательством Российской Федерации о некоммерческих организациях, структурного подразделения - отделения'}


addDF = nameSeparatedRows(addDF, names)

ind = addDF.index[addDF['part'] == '136'][0]
addDF.at[ind, 'part'] = '136ч.2'
addDF['part'] = addDF['part'].replace('168ч.1','168')

addDF['part'] = addDF['part'].replace({'^116\.1$': '116.1ч.1'}, regex=True)

firstCols = ['name', 'clause', 'part', 'totalConvictedMain']
addDF = rearrangeCols(addDF, firstCols)
addDF = sortTable(addDF)

Проверяем, что все переименовано.
Если ничего не печатается, то все переименовано.
Если остались не переименованные колонки, это иногда ок: мы не используем колонки, значения в которых легко вычисляются сложнением других колонок или повторяются в других файлах
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за отсутствием состава, события преступления, непричастностью к преступлению по основной статье
Число лиц, в отношении которых уголовные дела прекращены по иным основаниям по основной статье
Принудительные меры медицинского характера в отношении невменяемого Число лиц по основной статье
Проверяем, что все переименовано.
Если ничего не печатается, то все переименовано.
Если остались не переименованные колонки, это иногда ок: мы не используем колонки, значения в которых легко вычисляются сложнением других колонок или повторяются в других файлах
Число оправданных по основной статье
Число лиц, в отношении которых уголовные дела прекращены за

In [13]:
addDF[addDF.part.str.contains('116.1')]

,name,clause,part,totalConvictedMain,addTotalPersons,addTotalOffences,addAcquittalPersons,addAcquittalOffences,addDismissalPersons,addDismissalOffences,addDismissalOtherPersons,addDismissalOtherOffences,addUnfitToPleadOffences
32,"Нанесение побоев лицом, подвергнутым администр...",116.1,116.1ч.1,1612,380,747,2,2,5,5,361,460,9
33,"Нанесение побоев лицом, подвергнутым администр...",116.1,116.1ч.2,4168,834,1795,1,1,4,4,9,16,6


In [14]:
addDF.tail()

,name,clause,part,totalConvictedMain,addTotalPersons,addTotalOffences,addAcquittalPersons,addAcquittalOffences,addDismissalPersons,addDismissalOffences,addDismissalOtherPersons,addDismissalOtherOffences,addUnfitToPleadOffences
1047,"Нападение на лиц или учреждения, пользующиеся ...",360,360ч.2,1,0,0,0,0,0,0,0,0,0
1048,"Совершение вне пределов территории РФ взрыва, ...",361,361ч.1,0,0,0,0,0,0,0,0,0,0
1049,"Акт международного терроризма, финансирование ...",361,361ч.2,0,0,0,0,0,0,0,0,0,0
1050,"Деяния, предусмотренные частью первой настояще...",361,361ч.3,0,0,0,0,0,0,0,0,0,0
1051,Воинские преступления,Воинские преступления,Воинские преступления,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
checkTablesLen(mainDF, addDF, parametersDF)
checkTablesLen(mainDF, addDF)

ОК: длины таблиц № 10.3 и № 10.3.1 совпадают
ОК: длины таблиц № 10.3 и № 10-а совпадают
ОК: длины таблиц № 10.3 и № 10-а совпадают


In [16]:
checkNumbersBetweenForms(year, mainDF, addDF, parametersDF)

Не совпадает число осужденных по основной статье. 
Статья:  Воинские преступления 
10.3: 4409 
10-a: nan 
Год: 2022


In [17]:
addDF = addDF.drop(['clause', 'part', 'totalConvictedMain'], axis=1)
parametersDF = parametersDF.drop(['clause', 'part'], axis=1)
df = pd.concat([mainDF, parametersDF, addDF], axis=1)

In [18]:
df.head()

,clause,part,totalConvicted,primaryLifeSentence,primaryImprisonment,primarySuspended,primaryMilitaryDisciplinaryUnit,primaryArrest,primaryRestrain,primaryRestrictionsInMilitaryService,primaryCorrectionalLabour,primaryCommunityService,primaryForcedLabour,primaryDisqualification,primaryFine,primaryOther,exemptionTimeServedFromImprisonment,exemptionTimeServedOther,exemptionAmnestyFromImprisonment,exemptionAmnestyOther,exemptionOtherGroundsFromImprisonment,exemptionOtherGroundsOther,acquittal,dismissalAbsenceOfEvent,dismissalAmnesty,...,primaryFine1M,addFine5,addFine5_25,addFine25_100,addFine100_300,addFine300_500,addFine500_1M,addFine1M,primaryFineSum,addFineSum,dismissalCourtFine5,dismissalCourtFine5_25,dismissalCourtFine25_100,dismissalCourtFine100,dismissalCourtFineSum,name,addTotalPersons,addTotalOffences,addAcquittalPersons,addAcquittalOffences,addDismissalPersons,addDismissalOffences,addDismissalOtherPersons,addDismissalOtherOffences,addUnfitToPleadOffences
0,105,105ч.1,4461,0,4424,20,0,0,1,0,0,0,0,0,0,0,0,0,0,0,14,2,32,2,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Убийство,52,56,3,3,3,3,6,6,11
1,105,105ч.2,1225,33,1187,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,3,0,23,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Убийство при отягчающих обстоятельствах,39,117,24,25,2,2,12,13,7
2,106,106,22,0,14,2,0,0,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Убийство матерью новорожденного ребенка,0,0,0,0,0,0,0,0,0
3,107,107ч.1,27,0,1,0,0,0,15,0,3,0,0,0,0,3,1,3,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Убийство, совершенное в состоянии аффекта",0,0,0,0,0,0,0,0,0
4,107,107ч.2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Убийство в состоянии аффекта двух или более лиц,0,0,0,0,0,0,1,1,0


In [19]:
dfMelted = meltTable(df, year)

In [20]:
dfMelted.head(5)

,year,clause,part,name,parameter,value
0,2022,105,105ч.1,Убийство,primaryLifeSentence,0
1,2022,105,105ч.2,Убийство при отягчающих обстоятельствах,primaryLifeSentence,33
2,2022,106,106,Убийство матерью новорожденного ребенка,primaryLifeSentence,0
3,2022,107,107ч.1,"Убийство, совершенное в состоянии аффекта",primaryLifeSentence,0
4,2022,107,107ч.2,Убийство в состоянии аффекта двух или более лиц,primaryLifeSentence,0


In [21]:
dfMelted.to_pickle('2023.pkl')